# SOLUTION TO CHALLENGE LAB #1
**Important Note**: This problem and solution is from Leetcode(https://www.leetcode.com/).

In [4]:
class Solution(object):

    def __init__(self):
        self.valid_expressions = None
        self.min_removed = None

    def reset(self):
        self.valid_expressions = set()
        self.min_removed = float("inf")

    """
        string: The original string we are recursing on.
        index: current index in the original string.
        left: number of left parentheses till now.
        right: number of right parentheses till now.
        ans: the resulting expression in this particular recursion.
        ignored: number of parentheses ignored in this particular recursion.
    """
    def remaining(self, string, index, left_count, right_count, expr, rem_count):
        # If we have reached the end of string.
        if index == len(string):

            # If the current expression is valid. The only scenario where it can be
            # invalid here is if left > right. The other way around we handled early on in the recursion.
            if left_count == right_count:

                if rem_count <= self.min_removed:
                    # This is the resulting expression.
                    # Strings are immutable in Python so we move around a list in the recursion
                    # and eventually join to get the final string.
                    possible_ans = "".join(expr)

                    # If the current count of brackets removed < current minimum, ignore
                    # previous answers and update the current minimum count.
                    if rem_count < self.min_removed:
                        self.valid_expressions = set()
                        self.min_removed = rem_count

                    self.valid_expressions.add(possible_ans)    
        else:        

            current_char = string[index]

            # If the current character is not a parenthesis, just recurse one step ahead.
            if current_char != '(' and  current_char != ')':
                expr.append(current_char)
                self.remaining(string, index + 1, left_count, right_count, expr, rem_count)
                expr.pop()
            else:
                # Else, one recursion is with ignoring the current character.
                # So, we increment the ignored counter and leave the left and right untouched.
                self.remaining(string, index + 1, left_count, right_count, expr, rem_count + 1)

                expr.append(current_char)

                # If the current parenthesis is an opening bracket, we consider it
                # and increment left and  move forward
                if string[index] == '(':
                    self.remaining(string, index + 1, left_count + 1, right_count, expr, rem_count)
                elif right_count < left_count:
                    # If the current parenthesis is a closing bracket, we consider it only if we
                    # have more number of opening brackets and increment right and move forward.
                    self.remaining(string, index + 1, left_count, right_count + 1, expr, rem_count)

                expr.pop()

    def removeInvalidParentheses(self, s):
        """
        :type s: str
        :rtype: List[str]
        """

        # Reset the class level variables that we use for every test case.
        self.reset()

        # Recursive call
        self.remaining(s, 0, 0, 0, [], 0)
        return list(self.valid_expressions)

## Challenge: Time Effecient

In [2]:
class Solution:
    def removeInvalidParentheses(self, s: 'str') -> 'List[str]':
        def helper(s, i0, j0, p0, p1):
            '''
            count to see if there are more p1 than p0
                           forward :       )       (
                           backward:       (       )
            if found more p1, reconstruct partial s and recurse
            s : string being reconstructed
            i0: s[:i0] is correct already
            j0: when there is an extra p1, j0 is the first index where a previous p1 may be found and removed
            during recursiong, i0 and j0 are nondecreasing, so no repeated answers
            '''
            cnt = 0
            for i in range(i0, len(s)):
                if s[i] == p0:
                    cnt -= 1
                elif s[i] == p1:
                    cnt += 1
                if cnt > 0:
                    break
            else: # didn't find i
                r = s[::-1]
                if p0 == '(': # if forward. do backward
                    helper(r, 0, 0, p1, p0)
                else:
                    ret.append(r)
                return
            
            # found i. remove a redundant p1 and recurse
            for j in range(j0, i + 1):
                if s[j] == p1 and (j == j0 or s[j-1] != p1):
                    helper(s[:j] + s[j+1:], i, j, p0, p1)
            
        ret = []
        helper(s, 0, 0, '(', ')')
        return ret

# Challenge: Memory Effecient

In [4]:
from itertools import combinations

class Solution:
    def removeInvalidParentheses(self, s):
        """
        :type s: str
        :rtype: List[str]
        """
        # remove ')' in the beginning
        # remove '(' in the end
        # calculate the extra '(' or ')'
        cache = {}
        def isvalid(string):
            if string in cache:
                return cache[string]
            nleft = 0
            for ch in string:
                if ch == '(':
                    nleft += 1
                elif ch == ')':
                    nleft -= 1
                    if nleft < 0:
                        cache[string] = False
                        return False
            cache[string] = nleft == 0
            return nleft == 0

        s = (s.lstrip(')')).rstrip('(')
        left = 0
        right = 0
        leftIndex = []
        rightIndex = []
        for i in range(len(s)):
            if s[i] == '(':
                leftIndex.append(i)
                left += 1
            elif s[i] == ')':
                rightIndex.append(i)
                if left > 0:
                    left -= 1
                else:
                    right += 1
        result = set()
        for t1 in combinations(leftIndex, left):
            for t2 in combinations(rightIndex, right):
                candidate = ''.join([x for i, x in enumerate(str(s)) if i not in list(t1) and i not in list(t2)])
                if isvalid(candidate):
                    result.add(candidate)
        return list(result)